In [1]:
from datasets import load_dataset

# Load the WikiText-2 Raw dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# Display dataset information
# print(dataset)


# Print the first 5 examples from the training set
# Print the first 5 examples from the training set
for i in range(5):
    print(dataset["train"][i]["text"])


c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the dataset since wikitext couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'wikitext-2-raw-v1' at C:\Users\Lenovo\.cache\huggingface\datasets\wikitext\wikitext-2-raw-v1\0.0.0\b08601e04326c79dfdd32d625aee71d232d685c3 (last modified on Mon Feb 10 14:13:07 2025).



 = Valkyria Chronicles III = 


 Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . 

 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series 

In [2]:
from transformers import BertTokenizer, BertForPreTraining, DataCollatorForLanguageModeling
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

In [3]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

In [4]:
from transformers import BertForMaskedLM

# Load BERT model for masked language modeling
model = BertForMaskedLM.from_pretrained("bert-base-uncased")


BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

In [5]:
from torch.utils.data import DataLoader
from transformers import AdamW

# Create DataLoader
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))  # Use a subset for quick training
train_dataloader = DataLoader(train_dataset, batch_size=8, collate_fn=data_collator)

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
model.train()
for epoch in range(3):  # Train for 3 epochs
    for batch in train_dataloader:
        optimizer.zero_grad()
        
        # Print batch keys for debugging
        print("Batch keys:", batch.keys())
        
        # Forward pass
        outputs = model(**batch)
        
        # Print model outputs for debugging
        print("Model outputs:", outputs)
        
        # Check if loss is None
        if outputs.loss is None:
            raise ValueError("Loss is None. Check the model and input data.")
        
        # Backward pass
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        print(f"Epoch {epoch}, Loss: {loss.item()}")




c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Batch keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
Model outputs: MaskedLMOutput(loss=tensor(1.8472, grad_fn=<NllLossBackward0>), logits=tensor([[[ -6.6337,  -6.6730,  -6.6389,  ...,  -6.0923,  -5.9187,  -3.8064],
         [ -4.6448,  -4.4396,  -4.8204,  ...,  -4.9353,  -5.9100,  -1.9342],
         [ -8.0181,  -8.0007,  -7.9133,  ...,  -7.2930,  -7.1094,  -7.5799],
         ...,
         [-10.5829, -10.6881, -10.4276,  ...,  -9.1356,  -9.3133, -12.4075],
         [-10.8777, -10.9162, -10.9796,  ..., -10.8169, -10.8836, -11.0561],
         [-11.6262, -11.5655, -11.6841,  ...,  -8.7754,  -9.1516,  -7.2493]],

        [[ -7.1542,  -7.1808,  -7.0953,  ...,  -6.4928,  -6.4795,  -4.2352],
         [-11.4073, -11.7677, -11.5320,  ...,  -8.6528, -10.9984,  -8.1621],
         [ -7.7024,  -7.9178,  -7.8136,  ...,  -6.0641,  -7.6349,  -6.3140],
         ...,
         [ -7.1983,  -7.4538,  -7.2127,  ...,  -7.4741,  -6.2153,  -4.8457],
         [ -7.7356,  -7.9769,  

In [6]:
model.save_pretrained("./trained_bert_model")
tokenizer.save_pretrained("./trained_bert_model")

('./trained_bert_model\\tokenizer_config.json',
 './trained_bert_model\\special_tokens_map.json',
 './trained_bert_model\\vocab.txt',
 './trained_bert_model\\added_tokens.json')

In [7]:
# Load the trained model
model = BertForPreTraining.from_pretrained("./trained_bert_model")

# Test with a sample sentence
input_text = "The quick brown fox jumps over the [MASK] dog."
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model(**inputs)

# Decode the predicted token
predicted_token_id = outputs.prediction_logits.argmax(dim=-1)[0, 6]  # Position of [MASK]
predicted_token = tokenizer.decode(predicted_token_id)
print(predicted_token)  # Should predict "lazy" or similar

Some weights of BertForPreTraining were not initialized from the model checkpoint at ./trained_bert_model and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


over


In [8]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# Load the pre-trained BERT model and tokenizer
model = BertForMaskedLM.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Example sentence with a masked token
sentence = "The capital of France is [MASK]."

# Tokenize the input sentence
inputs = tokenizer(sentence, return_tensors="pt")

# Get the index of the masked token
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

# Perform the prediction
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Get the predicted token ID for the masked token
predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)

# Decode the predicted token ID to get the word
predicted_token = tokenizer.decode(predicted_token_id)

# Print the result
print(f"Original sentence: {sentence}")
print(f"Predicted word: {predicted_token}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Original sentence: The capital of France is [MASK].
Predicted word: paris


In [ ]:
sentences = [
    "The capital of france is [MASK].",
    "The largest planet in the solar system is [MASK].",
    "The author of 'Pride and Prejudice' is [MASK].",
    "The last day of the week is [MASK]." 
]

for sentence in sentences:
    inputs = tokenizer(sentence, return_tensors="pt")
    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
    predicted_token = tokenizer.decode(predicted_token_id)
    print(f"Original sentence: {sentence}")
    print(f"Predicted word: {predicted_token}\n")

Original sentence: The capital of france is [MASK].
Predicted word: paris

Original sentence: The largest planet in the solar system is [MASK].
Predicted word: pluto

Original sentence: The author of 'Pride and Prejudice' is [MASK].
Predicted word: unknown

Original sentence: The last day of the week is [MASK].
Predicted word: friday



In [13]:
# Save the model
model.save_pretrained("./my_bert_model")
tokenizer.save_pretrained("./my_bert_model")

# Load the model
model = BertForMaskedLM.from_pretrained("./my_bert_model")
tokenizer = BertTokenizer.from_pretrained("./my_bert_model")